<a href="https://colab.research.google.com/github/AchetaTangade/CartoonifyImage/blob/main/project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install streamlit --quiet

In [ ]:
import cv2
#import streamlit as st

In [ ]:
#%%writefile app.py
# the above command is a magic command to create a python file from colab
#it takes the entire code and puts it in app.py 
#since streamlit runs only on CLI(command line interface) using .py files
#import streamlit as st
#st.title("MY FIRST STREAMLIT WEB APP")
#st.subheader("Welcome to AI tutorial")
#st.write("RNSIT")

In [ ]:
!sudo apt install tesseract-ocr
!pip install pytesseract --quiet

Reading package lists... Done
Building dependency tree       
Reading state information... Done
tesseract-ocr is already the newest version (4.00~git2288-10f4998a-2).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 42 not upgraded.


In [ ]:
%%writefile app.py
import numpy as np
import streamlit as st
import pytesseract
from PIL import Image #python Imaging library, to open image, streamlit does not support cv2 directly

import cv2
#import numpy as np
import matplotlib.pyplot as plt


def edge_mask(img,line_size,blur_value):

  gray=cv2.cvtColor(img,cv2.COLOR_RGB2GRAY)
  gray_blur = cv2.medianBlur(gray,blur_value)

  edges=cv2.adaptiveThreshold(gray_blur,255, cv2.ADAPTIVE_THRESH_MEAN_C,cv2.THRESH_BINARY,line_size,blur_value)

  return edges

def color_quantization(img , k ):

    #transform img
    data=np.float32(img).reshape((-1,3))

    #Determine citeria                                             iteration,accuracy
    criteria = (cv2.TERM_CRITERIA_EPS+ cv2.TERM_CRITERIA_MAX_ITER, 20 , 0.001)

    #implementing k-means (clustering)
    ret , label , center = cv2.kmeans(data , k,None,criteria,10,cv2.KMEANS_RANDOM_CENTERS)
    
    center = np.uint8(center)

    result = center[label.flatten()]
    result = result.reshape(img.shape)

    return result
    


def cartoon(blurred,edges):
  c = cv2.bitwise_and(blurred,blurred,mask=edges)
  return c

def cartoonization(img):
  line_size=5

  blur_value = st.sidebar.slider('Sharpness of image (the lower the value, the sharper it is)', 1, 99, 25, step=2)
  k = st.sidebar.slider('Tune the color averaging effects (low: only similar colors will be smoothed, high: dissimilar color will be smoothed)', 2, 100,9,step=2)
  d = st.sidebar.slider('Tune the smoothness level of the image (the higher the value, the smoother the image)', 3, 99, 3, step=2)

  edges=edge_mask(img,line_size,blur_value)
  img_quantize=color_quantization(img , k )
  blurred = cv2.bilateralFilter(img_quantize,d,sigmaColor=200,sigmaSpace=200)
  cc=cartoon(blurred,edges)
  return cc

##################

st.set_option('deprecation.showfileUploaderEncoding',False) #to warning ignore
st.title("Cartoonify Images")  #print title and text
st.text("Upload the Image")

uploaded_file=st.sidebar.file_uploader("Choose an image...",type=['jpg','png','jpeg'])
if uploaded_file is not None:
  img=Image.open(uploaded_file)  #reads the image, similar cv2.imread
  image=np.array(img)
  st.image(image,caption="Uploaded Image",use_column_width=True) #displays the image in its actual size 
  st.write("")  #print blank space

  

  st.write("Cartoon image")   
  cartoon=cartoonization(image)
  st.image(cartoon,caption="Cartooned Image",use_column_width=True)
   


Writing app.py


In [ ]:
!streamlit run app.py & npx localtunnel --port 8501

2022-05-23 06:41:34.994 INFO    numexpr.utils: NumExpr defaulting to 2 threads.
npx: installed 22 in 3.668s
your url is: https://six-vans-send-35-221-27-10.loca.lt

  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.2:8501
  External URL: http://35.221.27.10:8501

